# Adding LLM Call

Let's make the first node as an "Agent" that can call OpenAI models using langchain.

### Install the libraries

In [1]:
!pip install langchain langchain_openai python-dotenv

Set the API keys for OpenAI

In [2]:
from dotenv import load_dotenv
import os

# load env variables from .env file
load_dotenv()

# accessing the env variables using os.environ
os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

# set the model as ChatOpenAI
model = ChatOpenAI(temperature=0)

## Step-1: Parse the city mentioned

extract the city that the user mentions in the query

In [4]:
# functions

def function_1(input_1):
    complete_query = f"""Your task is to provide only the city name based on the user query. 
    Nothing more, just the city name mentioned. 
    Following is the user query: {input_1}"""
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    return f"Agent Says: {input_2}"

In [5]:
# define a langchain graph

from langgraph.graph import Graph

workflow = Graph()

workflow.add_node('node_1', function_1)
workflow.add_node('node_2', function_2)

workflow.add_edge('node_1', 'node_2')

workflow.set_entry_point('node_1')
workflow.set_finish_point('node_2')

app = workflow.compile()

In [6]:
# invoking the graph with an input

# app.invoke("What's the temperature in Las Vegas")

In [7]:
# print the output at each stage

# input = "Hey there"
# for output in app.stream(input=input):
#     # stream() yields dictionaries with output keyed by node name
#     for key, val in output.items():
#         print(f"Output from node '{key}':")
#         print('---')
#         print(val)
#     print('\n---\n')

## Step-2: Adding a weather API call

- We create a new function which take's the city name and gives us the weather for that city.

- Open Weather Map is [integrated](https://python.langchain.com/docs/integrations/tools/openweathermap) into LangChain.

- we need to install pyowm, create an API key on the website of Open Weather Map (which takes a few hours to activate) and then run the cells below to get the weather of a given city.

In [8]:
!pip install pyowm

In [9]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper

# load env variables from .env file
load_dotenv()

# accessing the env variables using os.environ
os.environ['OPENWEATHERMAP_API_KEY'] = os.environ.get('OPENWEATHERMAP_API_KEY')

weather = OpenWeatherMapAPIWrapper()

In [10]:
# city_name = 'Las Vegas'
# weather_data = weather.run(city_name)
# print(f'The weather data at city [{city_name}] is: {weather_data}')

In [11]:
# functions

def function_1(input_1):
    complete_query = f"""Your task is to provide only the city name based on the user query. 
    Nothing more, just the city name mentioned. 
    Following is the user query: {input_1}"""
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    weather_data = weather.run(input_2)
    return weather_data

Integrate this into function 2 and call the function 2 as a "tool" or "weather_Agent"

In [12]:
# define a langchain graph

from langgraph.graph import Graph

workflow = Graph()

workflow.add_node('agent', function_1)
workflow.add_node('tool', function_2)

workflow.add_edge('agent', 'tool')

workflow.set_entry_point('agent')
workflow.set_finish_point('tool')

app = workflow.compile()

In [13]:
# app.invoke("What's the temperature in Las Vegas")

In [14]:
# print the output at each stage

# input = "What's the temperature in Las Vegas"
# for output in app.stream(input=input):
#     # stream() yields dictionaries with output keyed by node name
#     for key, val in output.items():
#         print(f"Output from node '{key}':")
#         print('---')
#         print(val)
#     print('\n---\n')

## Step-3: Adding another LLM Call to filter results

Here we only want the temperature but the current sentup gives us the full weather report.

So we can make another LLM call to filter data.

Also, we can use a dictionary and pass it between the nodes (we could also use list but dict makes it a bit easier)

In [15]:
# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. we will append new messages as we pass along nodes.
AgentState['messages'] = []

Our goal is to have this state filler as: {'messages': [HumanMessage, AIMessage, ...]}

Also now we need to modify our functions to pass info along the new AgentState

In [16]:
# functions

def function_1(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = f"""Your task is to provide only the city name based on the user query. 
    Nothing more, just the city name mentioned. 
    Following is the user query: {user_input}"""
    response = model.invoke(complete_query)
    state['messages'].append(response.content)  # Appending AIMessage response
    return state

def function_2(state):
    messages = state['messages']
    agent_response = messages[-1]
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(agent_response)
    state['messages'].append(weather_data)
    return state

def function_3(state):
    messages = state['messages']
    user_input = messages[0]
    available_info = messages[-1]
    complete_query = f"""Your task is to provide info concisely based on the user query and the available information from the internet.
    Following is the user query: "{user_input}", Available information: "{available_info}"
    """
    # return complete_query
    reponse = model.invoke(complete_query)
    return reponse.content

In [17]:
# define a langchain graph

from langgraph.graph import Graph

workflow = Graph()

workflow.add_node('agent', function_1)
workflow.add_node('tool', function_2)
workflow.add_node('responder', function_3)

workflow.add_edge('agent', 'tool')
workflow.add_edge('tool', 'responder')

workflow.set_entry_point('agent')
workflow.set_finish_point('responder')

app = workflow.compile()

In [18]:
inputs = {'messages': ["What's the temperature in Las Vegas"]}
app.invoke(inputs)

'The current temperature in Las Vegas is 10.95°C with scattered clouds. The high is 11.8°C and the low is 10.2°C. The temperature feels like 9.98°C with a wind speed of 13.86 m/s and 40% cloud cover.'

In [19]:
# print the output at each stage

input = {'messages': ["What's the temperature in Las Vegas"]}
for output in app.stream(input=input):
    # stream() yields dictionaries with output keyed by node name
    for key, val in output.items():
        print(f"Output from node '{key}':")
        print('---')
        print(val)
    print('\n---\n')

Output from node 'agent':
---
{'messages': ["What's the temperature in Las Vegas", 'Las Vegas']}

---

Output from node 'tool':
---
{'messages': ["What's the temperature in Las Vegas", 'Las Vegas', 'In Las Vegas, the current weather is as follows:\nDetailed status: scattered clouds\nWind speed: 13.86 m/s, direction: 50°\nHumidity: 72%\nTemperature: \n  - Current: 11.0°C\n  - High: 11.8°C\n  - Low: 10.2°C\n  - Feels like: 10.04°C\nRain: {}\nHeat index: None\nCloud cover: 40%']}

---

Output from node 'responder':
---
The current temperature in Las Vegas is 11.0°C with scattered clouds. The high temperature is 11.8°C and the low is 10.2°C. The humidity is at 72% with a wind speed of 13.86 m/s. The weather feels like 10.04°C with a cloud cover of 40%.

---

Output from node '__end__':
---
The current temperature in Las Vegas is 11.0°C with scattered clouds. The high temperature is 11.8°C and the low is 10.2°C. The humidity is at 72% with a wind speed of 13.86 m/s. The weather feels like 1

As we notice that there is a lot of appending to the array going on, we can make it a bit easier with the following

In [21]:
inputs = {"messages": ["how are you?"]}
app.invoke(inputs)

"I'm just a virtual assistant, so I don't have feelings or emotions. In Istanbul, the current weather is clear with a temperature of 6.95°C, a wind speed of 3.6 m/s, and a humidity of 79%."

We can make our agent smarter by saying only use the tool when needed, if not just respond back to the user.

1. Building a tool to the agent
2. Using a conditional edge to the agent with the option to either call the tool or not
3. Defining the criteria for the conditional edge as when to call the tool. We will define a funciton for this.

Let's start with the AgentState definition as mentioned a few calls above.

In [20]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

Building tool with agent (LLM Model) is made easy in langchain

In [22]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun

tools = [OpenWeatherMapQueryRun()]

model = ChatOpenAI(temperature=0, streaming=True)
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

Our modified function_1 now becomes as below. The reason is, we are passing the human message as state and appending response to the state. Also, our agent now has a tool bound to it, that it can use.

In [23]:
def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {'messages': [response]}

For funciton_2, we want to setup a tool and call it. It's made easy to invoke a tool in LangChain by using ToolInvocation and executing it with ToolExecutor. Then we respond back as a FunctionMessage so that our agent (node 1) knows that the tool was used and a response from the tool is available.

In [24]:
from langgraph.prebuilt import ToolInvocation, ToolExecutor
from langchain_core.messages import FunctionMessage
import json

tool_executor = ToolExecutor(tools)

def funciton_2(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool
    
    parsed_tool_input = json.loads(last_message.additional_kwargs['function_call']['arguments'])
    
    # we construct a ToolInvocation from the funciton_call and pass in the tool name and the expected str input for the OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs['function_call']['name'],
        tool_input=parsed_tool_input['__arg1']
    )
    
    # we call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    
    # we use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    
    # we return a list, because this well get added to the existing list
    return {'messages': [function_message]}


Finally, we define a function for the conditional edge, to help us figure out which direction to go (tool or user response)

We can benefit from the agent (LLM) response in LangChain, which has additional_kwargs to make a function_call with the name of the tool.

So our logic is, if function_call available in the additional_kwargs, then call tool if not then end the discussion and respond back to the user.

In [25]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    
    if 'function_call' in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"
    

Now with all of the changes above, our LangGraph app is modified as below:

In [26]:
# from langgraph.graph import Graph, END

# workflow = Graph()

# Or you could import StateGraph and pass AgentState to it
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node('agent', function_1)
workflow.add_node('tool', funciton_2)

# The conditional edge requires the following info below.
workflow.add_conditional_edges('agent', where_to_go, {
    # based on the return from where_to_go
    # if return is "continue" then we call the tool node.
    "continue": "tool",
    # otherwise we finish. END is a special node marking that the graph should finsh.
    "end": END
})

# we now add a normal edge from 'tool' to 'agent'.
# this means that if 'tool' is called, then it has to call the 'agent' next.
workflow.add_edge('tool', 'agent')

# basically, agent node has the option to call a tool node based on a condition,
# whereas tool node must call the agent in all cases based on this setup.
workflow.set_entry_point("agent")

app = workflow.compile()

We also pass the first message using HumanMessage component avaialable in langchain, makes it easy.

In [27]:
from langchain_core.messages import HumanMessage

inputs = {'messages': [HumanMessage(content='What is the temperature in las vegas')]}
app.invoke(inputs)

{'messages': [HumanMessage(content='What is the temperature in las vegas'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Las Vegas"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call'}),
  FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 13.86 m/s, direction: 50°\nHumidity: 65%\nTemperature: \n  - Current: 11.96°C\n  - High: 12.91°C\n  - Low: 10.93°C\n  - Feels like: 10.91°C\nRain: {}\nHeat index: None\nCloud cover: 75%', name='open_weather_map'),
  AIMessage(content='The current temperature in Las Vegas is 11.96°C. It is partly cloudy with a wind speed of 13.86 m/s and a humidity of 65%.', response_metadata={'finish_reason': 'stop'})]}

In [28]:
inputs = {'messages': [HumanMessage(content='What is the temperature in las vegas')]}

for output in app.stream(inputs):
    # stream() yields dictionaries with output keys by node name
    for key, val in output.items():
        print(f"Output from node '{key}':")
        print('---')
        print(val)
    print('\n---\n')

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Las Vegas"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call'})]}

---

Output from node 'tool':
---
{'messages': [FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 13.86 m/s, direction: 50°\nHumidity: 64%\nTemperature: \n  - Current: 12.06°C\n  - High: 12.91°C\n  - Low: 10.93°C\n  - Feels like: 10.99°C\nRain: {}\nHeat index: None\nCloud cover: 75%', name='open_weather_map')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='The current temperature in Las Vegas is 12.06°C. It feels like 10.99°C with broken clouds and a wind speed of 13.86 m/s.', response_metadata={'finish_reason': 'stop'})]}

---

Output from node '__end__':
---
{'messages': [HumanMessage(content='What is the temperature in las vegas'), AIMessage(content='', additional_kwa